In [ ]:
#H2

In [ ]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE, AdaptVQE
from qiskit_algorithms.optimizers import COBYLA,L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 0.7414  # Angstrom

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of H2 using four different methods:
    UHF, FCI, standard VQE, and AdaptVQE.
    """
    print(f"H₂ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"H 0 0 0; H 0 0 {BOND_LENGTH}"

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']:.8f} Hartree")
    
    # FCI Calculation
    # Note: For H2, RHF is sufficient as a starting point for FCI
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculations (VQE and AdaptVQE) ---
    
    # Set up the chemical problem for Qiskit
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz, which is used by both VQE methods
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )
    optimizer = L_BFGS_B(maxiter=1000)
    

    # Define the VQE instance that both algorithms will use
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    print(f"Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- AdaptVQE Calculation ---
    print("--- Running AdaptVQE Calculation ---")
    start_time = time.time()
    adapt_vqe = AdaptVQE(vqe_instance)
    # This temporary fix is needed for compatibility in some Qiskit versions
    adapt_vqe.supports_aux_operators = lambda: True
    
    solver_adapt = GroundStateEigensolver(mapper, adapt_vqe)
    result_adapt = solver_adapt.solve(problem)
    results['AdaptVQE'] = result_adapt.total_energies[0]
    print(f"AdaptVQE Energy: {results['AdaptVQE']:.8f} Hartree")
    print(f"AdaptVQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("          Final Energy Summary")
    print("="*50)
    print(f"  UHF Energy:         {results['UHF']} Hartree")
    print(f"  Standard VQE Energy:  {results['VQE']} Hartree")
    print(f"  AdaptVQE Energy:      {results['AdaptVQE']:.8} Hartree")
    print(f"  FCI (Exact) Energy:   {results['FCI']:.8f} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):   {abs(results['VQE'] - results['FCI']):.8f} Hartree")
    print(f"  Error (AdaptVQE vs FCI): {abs(results['AdaptVQE'] - results['FCI']):.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()


H₂ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-17 12:47:19

--- Running Classical Calculations ---
UHF Energy: -1.11668439 Hartree
FCI (Exact) Energy: -1.13727017 Hartree
Classical calculations finished in 0.44s

--- Running Standard VQE Calculation ---


/tmp/ipykernel_492559/208599009.py:77: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


Standard VQE Energy: -1.13727017 Hartree
VQE finished in 0.81s

--- Running AdaptVQE Calculation ---
AdaptVQE Energy: -1.13727017 Hartree
AdaptVQE finished in 0.43s


          Final Energy Summary
  UHF Energy:         -1.1166843870853413 Hartree
  Standard VQE Energy:  -1.1372701746608649 Hartree
  AdaptVQE Energy:      -1.1372702 Hartree
  FCI (Exact) Energy:   -1.13727017 Hartree
--------------------------------------------------
  Error (VQE vs FCI):   0.00000000 Hartree
  Error (AdaptVQE vs FCI): 0.00000000 Hartree


In [ ]:
"""RESULTS
H₂ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-09 12:10:39

--- Running Classical Calculations ---
UHF Energy: -1.11668439 Hartree
FCI (Exact) Energy: -1.13727017 Hartree
Classical calculations finished in 0.29s

--- Running Standard VQE Calculation ---
/tmp/ipykernel_959151/1457183300.py:77: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)
Standard VQE Energy: -1.13727017 Hartree
VQE finished in 1.95s

--- Running AdaptVQE Calculation ---
AdaptVQE Energy: -1.13727017 Hartree
AdaptVQE finished in 0.97s


==================================================
          Final Energy Summary
==================================================
  UHF Energy:         -1.11668439 Hartree
  Standard VQE Energy:  -1.13727017 Hartree
  AdaptVQE Energy:      -1.13727017 Hartree
  FCI (Exact) Energy:   -1.13727017 Hartree
--------------------------------------------------
  Error (VQE vs FCI):   0.00000000 Hartree
  Error (AdaptVQE vs FCI): 0.00000000 Hartree
=================================================="""

In [ ]:
#H3+

In [3]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE, AdaptVQE
from qiskit_algorithms.optimizers import SLSQP,L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "3-21g"
BOND_LENGTH = 0.8561
  # Angstrom, standard bond length for H3+

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of H3+ using four different methods:
    UHF, FCI, standard VQE, and AdaptVQE.
    """
    print(f"H₃⁺ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as an equilateral triangle
    r = BOND_LENGTH / np.sqrt(3)
    atom_str = (
        f"H {r:.4f} 0.0 0.0; "
        f"H {-r/2:.4f} {r * np.sqrt(3)/2:.4f} 0.0; "
        f"H {-r/2:.4f} {-r * np.sqrt(3)/2:.4f} 0.0"
    )

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    # Note: charge=1 for the H3+ cation
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=1, spin=0, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']:.8f} Hartree")
    
    # FCI Calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculations (VQE and AdaptVQE) ---
    
    # Set up the chemical problem for Qiskit, specifying charge=1
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=1, spin=0)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz, which is used by both VQE methods
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # Define the VQE instance that both algorithms will use
    vqe_instance = VQE(Estimator(), ansatz, L_BFGS_B())
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    print(f"Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- AdaptVQE Calculation ---
    print("--- Running AdaptVQE Calculation ---")
    start_time = time.time()
    adapt_vqe = AdaptVQE(vqe_instance)
    # This temporary fix is needed for compatibility in some Qiskit versions
    adapt_vqe.supports_aux_operators = lambda: True
    
    solver_adapt = GroundStateEigensolver(mapper, adapt_vqe)
    result_adapt = solver_adapt.solve(problem)
    results['AdaptVQE'] = result_adapt.total_energies[0]
    print(f"AdaptVQE Energy: {results['AdaptVQE']:.8f} Hartree")
    print(f"AdaptVQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("          Final Energy Summary")
    print("="*50)
    print(f"  UHF Energy:         {results['UHF']:.8f} Hartree")
    print(f"  Standard VQE Energy:  {results['VQE']:.8f} Hartree")
    print(f"  AdaptVQE Energy:      {results['AdaptVQE']:.8f} Hartree")
    print(f"  FCI (Exact) Energy:   {results['FCI']:.8f} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):   {abs(results['VQE'] - results['FCI']):.8f} Hartree")
    print(f"  Error (AdaptVQE vs FCI): {abs(results['AdaptVQE'] - results['FCI']):.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()


H₃⁺ Ground State Calculation Comparison for Basis: 3-21g
Started at: 2025-09-16 16:00:50

--- Running Classical Calculations ---
UHF Energy: -1.26997919 Hartree
FCI (Exact) Energy: -1.29989474 Hartree
Classical calculations finished in 0.52s



/tmp/ipykernel_399573/549558148.py:81: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, L_BFGS_B())


--- Running Standard VQE Calculation ---
Standard VQE Energy: -1.29989474 Hartree
VQE finished in 242.43s

--- Running AdaptVQE Calculation ---
AdaptVQE Energy: -1.29989474 Hartree
AdaptVQE finished in 377.88s


          Final Energy Summary
  UHF Energy:         -1.26997919 Hartree
  Standard VQE Energy:  -1.29989474 Hartree
  AdaptVQE Energy:      -1.29989474 Hartree
  FCI (Exact) Energy:   -1.29989474 Hartree
--------------------------------------------------
  Error (VQE vs FCI):   0.00000000 Hartree
  Error (AdaptVQE vs FCI): 0.00000000 Hartree


In [ ]:
"""H₃⁺ Ground State Calculation Comparison for Basis: 3-21g
Started at: 2025-09-09 12:13:15

--- Running Classical Calculations ---
UHF Energy: -1.26997919 Hartree
FCI (Exact) Energy: -1.29989474 Hartree
Classical calculations finished in 0.54s

/tmp/ipykernel_959151/2862908298.py:81: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, SLSQP())
--- Running Standard VQE Calculation ---
Standard VQE Energy: -1.29989455 Hartree
VQE finished in 153.49s

--- Running AdaptVQE Calculation ---
AdaptVQE Energy: -1.29989462 Hartree
AdaptVQE finished in 323.24s


==================================================
          Final Energy Summary
==================================================
  UHF Energy:         -1.26997919 Hartree
  Standard VQE Energy:  -1.29989455 Hartree
  AdaptVQE Energy:      -1.29989462 Hartree
  FCI (Exact) Energy:   -1.29989474 Hartree
--------------------------------------------------
  Error (VQE vs FCI):   0.00000019 Hartree
  Error (AdaptVQE vs FCI): 0.00000012 Hartree
==================================================

==================================================
           Final Energy Summary for H₃⁺
==================================================
  UHF Energy:          -1.23685874 Hartree
  Standard VQE Energy: -1.26126402 Hartree
  FCI (Exact) Energy:  -1.26126402 Hartree
--------------------------------------------------
  Error (VQE vs FCI):  0.00000000 Hartree
==================================================

"""

In [ ]:
"""OH-"""

In [1]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE, AdaptVQE
from qiskit_algorithms.optimizers import SLSQP,COBYLA,L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH =  0.9640  # Angstrom, equilibrium bond length for OH-

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of OH- using four different methods:
    UHF, FCI, standard VQE, and AdaptVQE.
    """
    print(f"OH⁻ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"O 0 0 0; H 0 0 {BOND_LENGTH}"

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    # Note: charge=-1 and spin=0 for the hydroxide anion
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=-1, spin=0, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']:.8f} Hartree")
    
    # FCI Calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculations (VQE and AdaptVQE) ---
    
    # Set up the chemical problem for Qiskit, specifying charge=-1
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=-1, spin=0)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz, which is used by both VQE methods
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # Define the VQE instance that both algorithms will use
    vqe_instance = VQE(Estimator(), ansatz, L_BFGS_B(maxiter=1500))
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    print(f"Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- AdaptVQE Calculation ---
    print("--- Running AdaptVQE Calculation ---")
    start_time = time.time()
    adapt_vqe = AdaptVQE(vqe_instance)
    # This temporary fix is needed for compatibility in some Qiskit versions
    adapt_vqe.supports_aux_operators = lambda: True
    
    solver_adapt = GroundStateEigensolver(mapper, adapt_vqe)
    result_adapt = solver_adapt.solve(problem)
    results['AdaptVQE'] = result_adapt.total_energies[0]
    print(f"AdaptVQE Energy: {results['AdaptVQE']:.8f} Hartree")
    print(f"AdaptVQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("          Final Energy Summary")
    print("="*50)
    print(f"  UHF Energy:         {results['UHF']:.8f} Hartree")
    print(f"  Standard VQE Energy:  {results['VQE']:.8f} Hartree")
    print(f"  AdaptVQE Energy:      {results['AdaptVQE']:.8f} Hartree")
    print(f"  FCI (Exact) Energy:   {results['FCI']:.8f} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):   {abs(results['VQE'] - results['FCI']):.8f} Hartree")
    print(f"  Error (AdaptVQE vs FCI): {abs(results['AdaptVQE'] - results['FCI']):.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()


OH⁻ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-17 10:25:10

--- Running Classical Calculations ---
UHF Energy: -74.05632813 Hartree
FCI (Exact) Energy: -74.08035479 Hartree
Classical calculations finished in 0.46s

--- Running Standard VQE Calculation ---


/tmp/ipykernel_456010/912677371.py:75: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, L_BFGS_B(maxiter=1500))


Standard VQE Energy: -74.08035479 Hartree
VQE finished in 1067.63s

--- Running AdaptVQE Calculation ---
AdaptVQE Energy: -74.08032158 Hartree
AdaptVQE finished in 543.30s


          Final Energy Summary
  UHF Energy:         -74.05632813 Hartree
  Standard VQE Energy:  -74.08035479 Hartree
  AdaptVQE Energy:      -74.08032158 Hartree
  FCI (Exact) Energy:   -74.08035479 Hartree
--------------------------------------------------
  Error (VQE vs FCI):   0.00000000 Hartree
  Error (AdaptVQE vs FCI): 0.00003322 Hartree


In [ ]:
"""OH⁻ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-09 12:36:29

--- Running Classical Calculations ---
UHF Energy: -74.06501711 Hartree
FCI (Exact) Energy: -74.09428808 Hartree
Classical calculations finished in 0.60s

/tmp/ipykernel_1000811/3289249938.py:75: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, SLSQP())
--- Running Standard VQE Calculation ---
Standard VQE Energy: -74.09428756 Hartree
VQE finished in 197.96s

--- Running AdaptVQE Calculation ---
AdaptVQE Energy: -74.09424738 Hartree
AdaptVQE finished in 244.18s


==================================================
          Final Energy Summary
==================================================
  UHF Energy:         -74.06501711 Hartree
  Standard VQE Energy:  -74.09428756 Hartree
  AdaptVQE Energy:      -74.09424738 Hartree
  FCI (Exact) Energy:   -74.09428808 Hartree
--------------------------------------------------
  Error (VQE vs FCI):   0.00000052 Hartree
  Error (AdaptVQE vs FCI): 0.00004070 Hartree
=================================================="""

  OH- Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-10 13:47:37

--- Running Classical Calculations ---
UHF Energy: -74.05739919 Hartree
FCI (Exact) Energy: -74.08172336 Hartree
Classical calculations finished in 0.59s

--- Running UCCSD VQE Calculation ---
/tmp/ipykernel_1328555/3984640952.py:81: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_uccsd = VQE(Estimator(), uccsd_ansatz, optimizer)
UCCSD VQE Energy: -74.08172083 Hartree
UCCSD VQE finished in 158.07s

--- Running 2-UpCCGSD VQE Calculation ---
/tmp/ipykernel_1328555/3984640952.py:111: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_kupccgsd = VQE(Estimator(), kupccgsd_ansatz, kupccgsd_optimizer)

In [ ]:
"""LiH"""

In [ ]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE, AdaptVQE
from qiskit_algorithms.optimizers import SLSQP, L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 1.5949  # Angstrom, experimental equilibrium bond length for LiH

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of LiH using four different methods:
    UHF, FCI, standard VQE, and AdaptVQE.
    """
    print(f"LiH Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"Li 0 0 0; H 0 0 {BOND_LENGTH}"

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    # Note: charge=0 and spin=0 for the neutral LiH molecule
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=0, spin=0, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']:.8f} Hartree")
    
    # FCI Calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculations (VQE and AdaptVQE) ---
    
    # Set up the chemical problem for Qiskit, specifying charge=0
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=0, spin=0)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz, which is used by both VQE methods
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # Define the VQE instance that both algorithms will use
    vqe_instance = VQE(Estimator(), ansatz, L_BFGS_B())
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    print(f"Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- AdaptVQE Calculation ---
    print("--- Running AdaptVQE Calculation ---")
    start_time = time.time()
    adapt_vqe = AdaptVQE(vqe_instance)
    # This temporary fix is needed for compatibility in some Qiskit versions
    adapt_vqe.supports_aux_operators = lambda: True
    
    solver_adapt = GroundStateEigensolver(mapper, adapt_vqe)
    result_adapt = solver_adapt.solve(problem)
    results['AdaptVQE'] = result_adapt.total_energies[0]
    print(f"AdaptVQE Energy: {results['AdaptVQE']:.8f} Hartree")
    print(f"AdaptVQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("          Final Energy Summary")
    print("="*50)
    print(f"  UHF Energy:         {results['UHF']:.8f} Hartree")
    print(f"  Standard VQE Energy:  {results['VQE']:.8f} Hartree")
    print(f"  AdaptVQE Energy:      {results['AdaptVQE']:.8f} Hartree")
    print(f"  FCI (Exact) Energy:   {results['FCI']:.8f} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):   {abs(results['VQE'] - results['FCI']):.8f} Hartree")
    print(f"  Error (AdaptVQE vs FCI): {abs(results['AdaptVQE'] - results['FCI']):.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()


In [ ]:
"""LiH Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-09 12:28:58

--- Running Classical Calculations ---
UHF Energy: -7.86202696 Hartree
FCI (Exact) Energy: -7.88240341 Hartree
Classical calculations finished in 0.62s

/tmp/ipykernel_999850/1448371883.py:75: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, SLSQP())
--- Running Standard VQE Calculation ---
Standard VQE Energy: -7.88239258 Hartree
VQE finished in 1835.60s

--- Running AdaptVQE Calculation ---
AdaptVQE Energy: -7.88194501 Hartree
AdaptVQE finished in 2442.26s


==================================================
          Final Energy Summary
==================================================
  UHF Energy:         -7.86202696 Hartree
  Standard VQE Energy:  -7.88239258 Hartree
  AdaptVQE Energy:      -7.88194501 Hartree
  FCI (Exact) Energy:   -7.88240341 Hartree
--------------------------------------------------
  Error (VQE vs FCI):   0.00001083 Hartree
  Error (AdaptVQE vs FCI): 0.00045840 Hartree
=================================================="""
LiH Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-10 13:35:41

--- Running Classical Calculations ---
UHF Energy: -7.86202696 Hartree
FCI (Exact) Energy: -7.88240341 Hartree
Classical calculations finished in 2.32s

--- Running UCCSD VQE Calculation ---
/tmp/ipykernel_1345152/349821736.py:81: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_uccsd = VQE(Estimator(), uccsd_ansatz, optimizer)
UCCSD VQE Energy: -7.88239258 Hartree
UCCSD VQE finished in 1706.90s

--- Running 2-UpCCGSD VQE Calculation ---
LiH Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-10 13:35:41

--- Running Classical Calculations ---
UHF Energy: -7.86202696 Hartree
FCI (Exact) Energy: -7.88240341 Hartree
Classical calculations finished in 2.32s

--- Running UCCSD VQE Calculation ---
/tmp/ipykernel_1345152/349821736.py:81: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_uccsd = VQE(Estimator(), uccsd_ansatz, optimizer)

In [ ]:
Calculating VQE energy for LiH at bond length 1.5949 Å using sto3g basis set...
VQE energy at standard bond length: -7.882175879993409 Hartree

Calculating LiH energy at bond length 1.5949 Å with basis sto-3g
VQE energy: -7.882175880011617 Hartree
Exact (FCI) energy: -7.882175990801266 Hartree
Energy difference: 1.1078964945454572e-07 mHartree

Trying with a larger basis set...
Calculating LiH energy at bond length 1.5949 Å with basis 6-31g

In [5]:
import numpy as np
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator

# Set to use Hamiltonian representation that's better for exact eigensolver
import qiskit_nature
qiskit_nature.settings.use_pauli_sum_op = False

def calculate_lih_energy(bond_length=1.5949, basis="sto-3g"):
    """
    Calculate VQE and exact (FCI) energies for LiH at a specific bond length
    
    Args:
        bond_length (float): Interatomic distance in Angstroms
        basis (str): Basis set name
        
    Returns:
        tuple: (VQE energy, exact energy)
    """
    print(f"Calculating LiH energy at bond length {bond_length} Å with basis {basis}")
    
    # Define molecule
    molecule = MoleculeInfo(
        symbols=["Li", "H"],
        coords=([0.0, 0.0, 0.0], [bond_length, 0.0, 0.0]),
        multiplicity=1,
        charge=0,
    )
    
    # Create driver with specified basis set
    driver = PySCFDriver.from_molecule(
        molecule,
        basis=basis,
    )
    
    # Run electronic structure calculation
    properties = driver.run()
    
    # Transform the problem (freeze core orbitals)
    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)
    
    # Extract problem parameters
    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals
    
    # Map to qubit operator
    mapper = ParityMapper(num_particles=num_particles)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    
    # Calculate exact energy
    numpy_solver = NumPyMinimumEigensolver()
    exact_result = numpy_solver.compute_minimum_eigenvalue(qubit_op)
    exact_energy = problem.interpret(exact_result).total_energies[0].real
    
    # Calculate VQE energy
    init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    var_form = UCCSD(
        num_spatial_orbitals, num_particles, mapper, initial_state=init_state
    )
    optimizer = SLSQP(maxiter=100)
    estimator = Estimator(approximation=True)
    
    vqe = VQE(
        estimator,
        var_form,
        optimizer,
        initial_point=[0] * var_form.num_parameters,
    )
    
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_energy = problem.interpret(vqe_calc).total_energies[0].real
    
    print(f"VQE energy: {vqe_energy:.8f} Hartree")
    print(f"Exact (FCI) energy: {exact_energy:.8f} Hartree")
    print(f"Energy difference: {(vqe_energy - exact_energy) * 1000:.8f} mHartree")
    
    # Return reference to problem for additional analysis if needed
    return vqe_energy, exact_energy, problem

# Calculate energy at standard LiH bond length with different basis sets
standard_length = 1.5949  # Standard LiH bond length in Angstroms

# Minimal basis
vqe_energy_sto3g, exact_energy_sto3g, _ = calculate_lih_energy(standard_length, "sto-3g")



Calculating LiH energy at bond length 1.5949 Å with basis sto-3g
VQE energy: -7.88217588 Hartree
Exact (FCI) energy: -7.88217599 Hartree
Energy difference: 0.00011078 mHartree


In [ ]:
"H2O"

In [ ]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of H2O using three different methods:
    UHF, FCI, and standard VQE.
    """
    print(f"💧 H₂O Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string (standard geometry)
    atom_str = "O 0.000000 0.000000 0.117790; H 0.000000 0.755453 -0.471161; H 0.000000 -0.755453 -0.471161"

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']:.8f} Hartree")
    
    # FCI Calculation
    # RHF is a suitable starting point for the FCI calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculation (VQE) ---
    
    # Set up the chemical problem for Qiskit
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )
    
    # Set the optimizer to L-BFGS-B
    optimizer = L_BFGS_B()

    # Define the VQE instance
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation (Optimizer: L-BFGS-B) ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    print(f"Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for H₂O")
    print("="*50)
    print(f"  UHF Energy:          {results['UHF']:.8f} Hartree")
    print(f"  Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"  FCI (Exact) Energy:  {results['FCI']:.8f} Hartree")
    print("-" * 50)
    # Compare quantum result to the exact FCI energy
    print(f"  Error (VQE vs FCI):  {abs(results['VQE'] - results['FCI']):.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

💧 H₂O Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-16 12:26:35

--- Running Classical Calculations ---
UHF Energy: -74.96314678 Hartree
FCI (Exact) Energy: -75.01277618 Hartree
Classical calculations finished in 0.52s



/tmp/ipykernel_352371/1080916399.py:77: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


--- Running Standard VQE Calculation (Optimizer: L-BFGS-B) ---


In [ ]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer


# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE, AdaptVQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g" # Using a minimal basis set for faster computation
BOND_LENGTH = 1.087  # Angstrom, standard C-H bond length for Methane

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of CH4 using four different methods:
    UHF, FCI, standard VQE, and AdaptVQE.
    """
    print(f"CH₄ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's tetrahedral geometry
    # Coordinates are derived from the C-H bond length
    val = BOND_LENGTH / np.sqrt(3)
    atom_str = (
        f"C 0.0000 0.0000 0.0000; "
        f"H {val:.4f} {val:.4f} {val:.4f}; "
        f"H {val:.4f} {-val:.4f} {-val:.4f}; "
        f"H {-val:.4f} {val:.4f} {-val:.4f}; "
        f"H {-val:.4f} {-val:.4f} {val:.4f}"
    )

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    # Note: charge=0 for the neutral CH4 molecule
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=0, spin=0, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']:.8f} Hartree")
    
    # FCI Calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculations (VQE and AdaptVQE) ---
    
    # Set up the chemical problem for Qiskit, specifying charge=0
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=0, spin=0)
    problem_full = driver.run()
    
    # Freeze the 1s core orbital of the Carbon atom to reduce problem size
    transformer = FreezeCoreTransformer(freeze_core=True)
    problem = transformer.transform(problem_full)
    
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz, which is used by both VQE methods
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # Define the VQE instance that both algorithms will use
    vqe_instance = VQE(Estimator(), ansatz, SLSQP())
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    # Use problem.interpret to get the final energy including frozen core contributions
    final_vqe_result = problem.interpret(result_vqe)
    results['VQE'] = final_vqe_result.total_energies[0]
    print(f"Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- AdaptVQE Calculation ---
    print("--- Running AdaptVQE Calculation ---")
    start_time = time.time()
    adapt_vqe = AdaptVQE(vqe_instance)
    # This temporary fix is needed for compatibility in some Qiskit versions
    adapt_vqe.supports_aux_operators = lambda: True
    
    solver_adapt = GroundStateEigensolver(mapper, adapt_vqe)
    result_adapt = solver_adapt.solve(problem)
    # Use problem.interpret to get the final energy
    final_adapt_result = problem.interpret(result_adapt)
    results['AdaptVQE'] = final_adapt_result.total_energies[0]
    print(f"AdaptVQE Energy: {results['AdaptVQE']:.8f} Hartree")
    print(f"AdaptVQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("               Final Energy Summary for CH₄")
    print("="*50)
    print(f"  UHF Energy:          {results['UHF']:.8f} Hartree")
    print(f"  Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"  AdaptVQE Energy:     {results['AdaptVQE']:.8f} Hartree")
    print(f"  FCI (Exact) Energy:  {results['FCI']:.8f} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):    {abs(results['VQE'] - results['FCI']):.8f} Hartree")
    print(f"  Error (AdaptVQE vs FCI): {abs(results['AdaptVQE'] - results['FCI']):.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

CH₄ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-16 14:20:30

--- Running Classical Calculations ---
UHF Energy: -39.72680917 Hartree
FCI (Exact) Energy: -39.80568529 Hartree
Classical calculations finished in 1.14s



/tmp/ipykernel_373981/3420063857.py:90: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, SLSQP())


--- Running Standard VQE Calculation ---


In [2]:
!pip list|grep qiskit

qiskit                  1.1.0
qiskit-algorithms       0.4.0
qiskit-nature           0.7.2


In [3]:
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 21.5 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 35.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 44.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]
